---
title: "Travel Time and Spatial Patterns in road Infrastructure"
author: "Alejandro Ouslan"
format:
  revealjs:
    logo: assets/logo-aeasp.jpeg
    footer: "AEASP 2024"
---


## Research question

- We are looking at the relationship between road length and average comute time on a particular puma
- How does the changes in road lenth of a particular PUMA affect the average comute time of neighboring PUMAs?


In [ ]:
import os
os.chdir("..")
import plotly.express as px
import pandas as pd
import geopandas as gpd
puma = gpd.read_file("data/interim/puma.gpkg", engine="pyogrio")
df_roads = pd.read_parquet("data/processed/roads.parquet")
df_acs = pd.read_parquet("data/processed/acs.parquet")
puma.sort_values("puma_id", inplace=True)
puma.reset_index(drop=True, inplace=True)
df_acs["puma_id"] = df_acs["state"].astype(str).str.zfill(2) + df_acs["PUMA"].astype(str).str.zfill(5)
df_acs = df_acs[(df_acs["year"] >= 2012) & (df_acs["race"] == "ALL") & (df_acs["sex"] == 3)].reset_index(drop=True)
master_df = df_acs.merge(df_roads, on=["puma_id", "year"], how="left")
master_df = master_df[["year", "puma_id", "avg_time", "length"]].sort_values(by=["year", "puma_id"], ascending=True).reset_index(drop=True)
master_df["length"] = master_df["length"] / 1000
master_df = master_df[master_df["state"] == "06"]

## Maps 


In [ ]:
fig = px.choropleth_mapbox(master_df,
                            geojson=puma.geometry,
                            locations=master_df.index,
                            color="avg_time",
                            center={"lat": 37.0902, "lon": -95.7129},
                            mapbox_style="carto-positron",
                            range_color=[0, 45],
                            color_continuous_scale="Viridis",
                            zoom=3)
fig.show()

## Methods 

- 2351 PUMAs (statistical geographic areas containing no fewer than 100,000 people each.)
- 8 years of data
- 18808 observations
- Spatial Lag Panel Model with Fixed Effects:
$$y_{it} = \rho \sum_{j=1}^N w_{ij} y_{jt}  +  x_{it} \beta  +  \mu_i  +  e_{it}$$

## Weights 

![weights](assets/queens.png)

## Results Pt.1

| Variable | Left | Right | Center |
|---------|:-----|------:|:------:|
| Spatial Average Weight      | 12   |    12 |   12   |
| 123     | 123  |   123 |  123   |
| 1       | 1    |     1 |   1    |

: Demonstration of pipe table syntax


## Results Pt.2

::: {#tbl-panel layout-ncol=2}
| Variable | All | Black |
|------|------|------|
| Spatial splilover | B    | C    |
| Road Length | F    | G    |
| A    | G    | G    |

: Male {#tbl-first}

| Col1 | Col2 | Col3 |
|------|------|------|
| A    | B    | C    |
| E    | F    | G    |
| A    | G    | G    |

: Female {#tbl-second}

Main Caption
:::

See @tbl-panel for details, especially @tbl-second.

## Conclusion & clossing remarks